In [1]:
using Pkg
Pkg.activate(".")


  Activating project at `~/Research/SISAP2023/sisap2023`


In [5]:
]develop ../../InvertedFiles.jl

   Resolving package versions...
    Updating `~/Research/SISAP2023/sisap2023/Project.toml`
  [b20bd276] ~ InvertedFiles v0.5.1 ⇒ v0.5.2 `../../InvertedFiles.jl`
    Updating `~/Research/SISAP2023/sisap2023/Manifest.toml`
  [b20bd276] ~ InvertedFiles v0.5.1 ⇒ v0.5.2 `../../InvertedFiles.jl`


In [12]:
]status

Status `~/Research/SISAP2023/sisap2023/Project.toml`
  [336ed68f] CSV v0.10.9
  [a93c6f00] DataFrames v1.5.0
  [f67ccb44] HDF5 v0.16.14
  [ac1192a8] HypertextLiteral v0.9.4
  [b20bd276] InvertedFiles v0.5.1
  [033835bb] JLD2 v0.4.31
  [6f286f6a] MultivariateStats v0.10.1
  [98572fba] Parquet2 v0.2.10
  [053f045d] SimilaritySearch v0.10.3
  [7f6f6c8a] TextSearch v0.14.0


In [2]:
#=Pkg.add([
        PackageSpec(name="SimilaritySearch"),
        PackageSpec(name="TextSearch"),
        PackageSpec(name="JLD2"),
        PackageSpec(name="CSV"),
        PackageSpec(name="Parquet2"),
        PackageSpec(name="DataFrames"),
        PackageSpec(name="HypertextLiteral"),
])
=#
# Pkg.instantiate()

In [6]:
using SimilaritySearch, TextSearch, Parquet2, DataFrames, CSV, HypertextLiteral, JLD2, LinearAlgebra
using Downloads: download


[ Info: Precompiling TextSearch [7f6f6c8a-3b03-11e9-223d-e7d88259bd6c]
[ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]


In [19]:
dbname = "SISAP23-Challenge/laion2B-en-clip768-n=10M.h5"
indexname = "index.laion2b-en-clip768-n=10M.jld2"

"index.laion2b-en-clip768-n=10M.jld2"

In [18]:
;ls

binwalk-models
Creating PCA models.ipynb
demo10M.ipynb
index.laion2b-en-clip768-n=10M.jld2
laion-gold-standards.jl
laion-hamming-projection.jl
laion-pca-projection.jl
LICENSE
Manifest.toml
meta-10M.parquet
model-pca32-laion2B-10M.jld2
model-pca96-laion2B-10M.jld2
Projecting with PCA models.ipynb
Project.toml
README.md
SISAP23-Challenge


In [20]:
db = jldopen(dbname) do f
    X = Float32.(f["emb"])
    for c in eachcol(X)
        normalize!(c)
    end
    
    StrideMatrixDatabase(X)
end

StrideMatrixDatabase{StrideArraysCore.StrideArray{Float32, 2, (1, 2), Tuple{Int64, Int64}, Tuple{Nothing, Nothing}, Tuple{Static.StaticInt{1}, Static.StaticInt{1}}, Matrix{Float32}}}(Float32[-0.0019801862 0.0572678 … -0.020251254 0.026359847; 0.043159664 0.05241408 … -0.0052154227 0.068279326; … ; -0.009286653 0.018865405 … 0.02823272 0.023171648; -0.024937373 -0.008127693 … -0.025043186 0.02204281])

In [21]:
if isfile(indexname)
    G, _ = loadindex(indexname, db; staticgraph=true)
else
    dist = NormalizedCosineDistance()
    G = SearchGraph(; db, dist, verbose=false)
    index!(G)
    optimize!(G, MinRecall(0.9))
    saveindex(indexname, G; store_db=false)
end

(SearchGraph{NormalizedCosineDistance, StrideMatrixDatabase{StrideArraysCore.StrideArray{Float32, 2, (1, 2), Tuple{Int64, Int64}, Tuple{Nothing, Nothing}, Tuple{Static.StaticInt{1}, Static.StaticInt{1}}, Matrix{Float32}}}, SimilaritySearch.AdjacencyLists.StaticAdjacencyList{UInt32}, BeamSearch}
  dist: NormalizedCosineDistance NormalizedCosineDistance()
  db: StrideMatrixDatabase{StrideArraysCore.StrideArray{Float32, 2, (1, 2), Tuple{Int64, Int64}, Tuple{Nothing, Nothing}, Tuple{Static.StaticInt{1}, Static.StaticInt{1}}, Matrix{Float32}}}
  adj: SimilaritySearch.AdjacencyLists.StaticAdjacencyList{UInt32}
  hints: Array{Int32}((165,)) Int32[785, 845, 1416, 2220, 2504, 2626, 2686, 2838, 2954, 3026  …  20115, 20143, 20146, 20295, 20480, 20500, 20518, 20520, 20564, 20759]
  search_algo: BeamSearch
  len: Base.RefValue{Int64}
  verbose: Bool true
, nothing)

In [23]:
size(db.matrix)

(768, 10120191)

In [8]:
#jldopen(f->f["pca96"], "laion2B-en-pca96-n=10M.h5")

In [25]:
D = DataFrame(Parquet2.Dataset("SISAP23-Challenge/meta-10M.parquet"), copycols=false)

Row,caption,NSFW,LICENSE,url
,String,String,String?,String
1,ONA Prince Street Camera Messenger Bag - Field Tan,UNLIKELY,?,https://cdn.shopify.com/s/files/1/0253/1179/products/TBPhotography-6754_small.jpg?v=1571438581
2,"One bed - three looks! You can keep the bones of your bedroom the same - your bed and major storage pieces, and completely transform the look of your the space with a switch of bedding, lighting and accessories.",UNLIKELY,?,https://i.pinimg.com/236x/e6/2d/ce/e62dce0011d491e2eba1b19e03ff33e3--summer-bedroom-ikea-bedroom.jpg
3,White flag at end of stick held by an arm against a partly cloudy sky.,UNLIKELY,?,https://g.foolcdn.com/editorial/images/472551/surrender-gettyimages-160054088_large.jpg
4,Our Complete Ikea Kitchen Remodel 8 Most Helpful Ideas,UNLIKELY,?,https://i1.wp.com/www.apieceofrainbow.com/wp-content/uploads/2019/07/Farmhouse-IKEA-Kitchen-Remodel-before-after-DIY-small-kitchen-design-remodeling-renovation-colorful-Modern-bohemian-boho-anthropologie-home-apieceofrainbow-1e.jpg?q=w%3D39%26strip%3Dall%26quality%3D24
5,Durham Bulls Seasonal Job Fair,UNLIKELY,?,https://www.capitolbroadcasting.com/wp-content/uploads/2015/01/job-fair-seasonal-150x150.jpg
6,"32 Living Room Library Design, Library/Living Room Foxcroft ... Modern Home Office Design Ideas Best 25 Offices For Small Space Interior Library Pictures Mens Study Room Webbkyrkancom Simple Nice With Dark Wooden Table Study Rooms Ideas On Pinterest Desk Families It Decorating Entrancing Home Office",UNLIKELY,?,https://t0.gstatic.com/images?q=tbn:ANd9GcRKpOQU_OhubsM65QRYDIVFPw2X6K_WBbzGnRyp-06_fpBKc_8n
7,Cover for The judge's list,UNLIKELY,?,https://contentcafe2.btol.com/ContentCafe/Jacket.aspx?UserID=NPI86627&Password=CC59937&Return=T&Type=S&Value=0385546025
8,Chapeaux Paintings - Sunday Hat in Sint Maarten by Patty Meotti,UNLIKELY,?,http://images.fineartamerica.com/images-small/sunday-hat-in-sint-maarten-patty-meotti.jpg
9,"""""""Down the Road"""" original fine art by Jo Allebach""",UNLIKELY,?,http://fineartimages.dailypaintworks.com/39c6bbb0-691c-4dc9-8aa3-a0e94eec6d5f.jpg?imageVersion=


In [ ]:
invfilename = "invfile-captions-laion2B-en-10M.jld2"
#isfile(invfilename) && rm(invfilename)
if isfile(invfilename)
    invfile = load(invfilename, "invfile")
else
    invfile = BM25InvertedFile(TextConfig(), rand(D.caption, 300_000)) do t
        3 <= t.ndocs <= 50_000
    end

    @time append_items!(invfile, D.caption)
    jldsave(invfilename; invfile)
end

In [30]:
function show_results(res)
    L = []

    for (i, p) in enumerate(res)
        r = D[p.id, :]

        push!(L, @htl("""
        <div style="width: 12%; display: inline-block; vertical-align: top; ">
                <img src="$(r.url)" style="width: 95%;"/>
                
                <span style="background-color: rgb(200, 200, 222);">$i - id: $(p.id) - w: $(round(p.weight; digits=2))</span>
                <span>$(r.caption)</span>
        </div>
            """))
    end

    @htl("""
        <div>$L</div>
    """)
end

function search_text(invfile::BM25InvertedFile, qtext::AbstractString, k)
    @time R = search(invfile, qtext, getknnresult(k))
    show_results(R.res)
end

search_text(qtext::AbstractString, k=16) = search_text(invfile, qtext, k)

function search_emb(G::SearchGraph, q, k)
    @time R = search(G, q, getknnresult(k))
    show_results(R.res)
end

search_emb(q, k=16) = search_emb(G, q, k)

search_emb (generic function with 3 methods)

In [37]:
search_text("guacamole")

  0.000311 seconds (3.35 k allocations: 67.883 KiB)


<div><div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://eclecticrecipes.com/wp-content/uploads/2009/08/guac-135x135.png" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">1 - id: 794436 - w: -24.61</span>
        <span>Perfect Guacamole @EclecticEveryday</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://i1.wp.com/www.cutsandcrumbles.com/wp-content/uploads/2016/04/Cheesy-Guacamole-8.jpg?resize=400%2C367" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">2 - id: 1183444 - w: -24.61</span>
        <span>Cheesy Guacamole</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://www.livinglou.com/wp-content/uploads/2013/02/5-Ingredient-Guacamole-150x150.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">3 - id: 4488646 - w: -24.61</span>
        <span>5 Ingredient Guacamole</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://media-cdn.tripadvisor.com/media/photo-l/13/2a/03/0e/guacamole-and-grasshoppers.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">4 - id: 5613693 - w: -24.61</span>
        <span>Guacamole and grasshoppers</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://www.megamexfoodservice.com/wp-content/uploads/2018/09/37652_Wholly-Guacamole_Homestyle_-Front_R.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">5 - id: 6456586 - w: -24.61</span>
        <span>37652_Wholly Guacamole_Homestyle_ Front_R</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://theproofofthepuddingdotnet.files.wordpress.com/2014/03/img_1190.jpg?w=642&amp;" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">6 - id: 6680829 - w: -24.61</span>
        <span>Homemade guacamole</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://mt-backend-familycircle-environment-contentbucket-1qlli1qnqqj9z.s3.amazonaws.com/s3fs-public/styles/398x398/public/recipe/550_RU266877.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">7 - id: 8694599 - w: -24.61</span>
        <span>Pomegranate Guacamole</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://sep.yimg.com/ay/mex-grocer/wholly-guacamole-minis-classic-mild-2-oz-29.png" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">8 - id: 4961550 - w: -24.33</span>
        <span>Guacamole by Wholly Organic Guacamole Brand Minis 2 oz - image 2</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://cdn.trendhunterstatic.com/phpthumbnails/271/271469/271469_1_468.jpeg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">9 - id: 249175 - w: -23.98</span>
        <span>Citrusy Guacamole Recipes - This Delicious Recipe for Guacamole Also Incorporates Grapefruit</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://i1.wp.com/mommysmemorandum.com/wp-content/uploads/wholly-guacamole-spider-web-dip-with-other-wholly-guacamole-products.jpg?fit=700%2C439&amp;ssl=1" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">10 - id: 2825807 - w: -23.98</span>
        <span>wholly guacamole spider web dip with other wholly guacamole products</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img sr

In [38]:
search_emb(database(G, 1875558))

  0.006626 seconds (5 allocations: 4.297 KiB)


<div><div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://i2.wp.com/www.greatgrubdelicioustreats.com/wp-content/uploads/2015/04/036.jpg?resize=550%2C825&amp;" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">1 - id: 1875558 - w: 0.0</span>
        <span>Fresh Spicy Guacamole</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://media2.popsugar-assets.com/files/thumbor/TOGaoUh6nRiFYKkvspFsdiCRFB0=/fit-in/2048xorig/2014/05/21/918/n/1922195/de70731df31394bb_Guacamole/i/Chips-Guacamole.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">2 - id: 3058736 - w: 0.05</span>
        <span>Chips and Guacamole</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://sugarspunrun.com/wp-content/uploads/2020/07/Guacamole-recipe-2-of-8.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">3 - id: 8655183 - w: 0.05</span>
        <span>guacamole in white bowl surrounded by corn chips</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://happyhooligans.ca/wp-content/uploads/2013/12/IMG_7593-2.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">4 - id: 7971711 - w: 0.06</span>
        <span>homemade guacamole in blue bowl</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://andrewzimmern.com/wp-content/uploads/Guacamole-800x600.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">5 - id: 1712832 - w: 0.06</span>
        <span>Andrew Zimmern&apos;s recipe for guacamole</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://www.eatingonadime.com/wp-content/uploads/2016/02/guacamole-8.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">6 - id: 6488560 - w: 0.06</span>
        <span>If you love fresh guacamole then you need to try this delicious simple guacamole recipe. This is the easiest homemade guacamole recipe, ever!</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://homemadehooplah.com/wp-content/uploads/2016/06/homemade-guacamole-2-700x1048.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">7 - id: 7449719 - w: 0.06</span>
        <span>Homeamde Guacamole! Classic guacamole that&apos;s ready in 10 minutes and has an extra hint of garlic for a bold flavor. Perfect as a party dip or adding to other southwestern favorites. Plus, make it as mild or as spicy as you like!   HomemadeHooplah.com</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://mrscriddleskitchen.com/wp-content/uploads/2015/06/guaca-1024x683.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">8 - id: 2566664 - w: 0.06</span>
        <span>A beautiful easy Homemade Guacamole you will love!</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://images.media-allrecipes.com/userphotos/848092.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">9 - id: 1848702 - w: 0.07</span>
        <span>Traditional Mexican Guacamole</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://static.wixstatic.com/media/2d24c9_86c9a910480c4e77b59c6095d59d442a~mv2_d_2000_1324_s_2.jpeg/v1/fit/w_1000%2Ch_1000%2Cal_c%2Cq_80/file.png" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">10 - id: 3162709 - w: 

In [43]:
search_emb(database(G, 88899))

  0.055591 seconds (7 allocations: 28.359 KiB)


<div><div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://www.newvideo.com/wp-content/themes/hybrid-newvideo/tools/timthumb.php?src=/wp-content/uploads/2012/10/itunes_inthemirrorofmayaderen.jpg&amp;w=120&amp;zc=1&amp;q=95" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">1 - id: 88899 - w: 0.0</span>
        <span>In the Mirror of Maya Deren</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://re-voir.com/shop/77-home_default/experimental-films-maya-deren.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">2 - id: 8431480 - w: 0.25</span>
        <span>Experimental films /DVD</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://blob.cede.ch/catalog/17103000/17103533_1_91.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">3 - id: 8069758 - w: 0.26</span>
        <span>The Maya Deren Collection</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://s3-eu-central-1.amazonaws.com/plusbog-covers/large/9780349006215.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">4 - id: 1739989 - w: 0.3</span>
        <span>Maya Angelou: The Complete Poetry</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://jwa.org/sites/default/files/styles/bio_indeximage_thumb/public/mediaobjects/maya_deren.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">5 - id: 8510444 - w: 0.31</span>
        <span>&quot;Maya Deren in &quot;&quot;Meshes of the Afternoon,&quot;&quot; 1943&quot;</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://img.reelgood.com/content/movie/ad39c4dd-6ff8-4ead-b717-f56eec37ddd2/poster-780.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">6 - id: 6661456 - w: 0.32</span>
        <span>Maya Angelou: And Still I Rise Poster</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://www.thatsnotus.com/g/023-slouching-towards-bethlehem-essay-example-joan-didion-728x1142.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">7 - id: 752588 - w: 0.32</span>
        <span>023 Slouching Towards Bethlehem Essay Example Joan Didion Singular Essays Collections On Santa Ana Winds Amazon 728</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://imgv2-2-f.scribdassets.com/img/word_document/161661420/149x198/f64f0b169e/1544433869?v=1" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">8 - id: 4134450 - w: 0.33</span>
        <span>Amy Winehouse</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://w10.hdonline.eu/images/tt1655420/poster.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">9 - id: 1429987 - w: 0.33</span>
        <span>My Week with Marilyn</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://orion-uploads.openroadmedia.com/lg_af6d85-jane-eyre-vintage.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">10 - id: 4252278 - w: 0.33</span>
        <span>Jane Eyre by Charlotte Bronte</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://i2.wp.com/inspire4u.co.uk/wp-content/uploads/2021/06/sylviaplathjournals.jpgw680ssl1.jpeg?fit=194%2C300&amp;ssl=1" style="width: 95%;"/>
    

In [35]:
search_text("picasso")

  0.000938 seconds (13.86 k allocations: 246.945 KiB)


<div><div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://cdn.shopify.com/s/files/1/2584/1070/products/picasso-and-picasso-petrina-easton_eb0d4d2d-4baa-4085-a1e3-6cfb126829cc_2048x2048.jpg?v=1515713857" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">1 - id: 2910582 - w: -24.17</span>
        <span>Picasso And Picasso - Phone Case</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://tse1-2.mm.bing.net/th?id=OIP.c7Jo90ZDr3Xn1h_4CW15hQHaI4" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">2 - id: 276649 - w: -24.06</span>
        <span>picasso paintings of therese best 25 pablo picasso ideas on picasso</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://tse1.mm.bing.net/th?id=OIP.MgBbz0gtW3iJNRhI6dfvOwHaFj" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">3 - id: 1088261 - w: -23.77</span>
        <span>picasso paintings guernica meaning guernica by pablo picasso</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://tse1.mm.bing.net/th?id=OIP.SzAqg7oi-XALwycDk_OcVQHaKU" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">4 - id: 1415158 - w: -23.77</span>
        <span>picasso paintings buy picasso paintings 2018 dr</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://ecx.images-amazon.com/images/P/1573229539.01._SL160_SCLZZZZZZZ__.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">5 - id: 1548318 - w: -23.77</span>
        <span>Picasso, Marina: Picasso, My Grandfather</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://www.michelfillion.com/img/photos/picasso_122b.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">6 - id: 4005876 - w: -23.77</span>
        <span>PICASSO : picasso-profil-homme-etching</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://www.ming-gallery.com/img_small/TN_PIC0133.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">7 - id: 8705027 - w: -23.77</span>
        <span>Picasso, PIC0133 Picasso Painting Art Reproduction</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://ctl.s6img.com/society6/img/ZDYcvOtZRTTcsrhX3jIdPIb63EQ/w_200,h_200/bags/small/close/~artwork/s6-0019/a/6939387_15826991.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">8 - id: 9560756 - w: -23.77</span>
        <span>picasso Tote Bags featuring Picasso by Mitja Bokun</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://dcqohhuh7tdzn.cloudfront.net/9780/9783829029780_medium.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">9 - id: 2720965 - w: -23.4</span>
        <span>Picasso : Life and Work, Picasso, Pablo, 3829029780</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://www.delcampe.net/static/img_thumb/auction/000/531/327/797_001.jpg?v=192" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">10 - id: 3970438 - w: -23.4</span>
        <span>GUINEA 2018 - Pablo Picasso. Official Issue - Picasso</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://i.pinimg.com/236x/96/64/2e/96642e20600a220c4769819299fe57bc--picasso-blue-period-blue-dog.jpg" style="width: 95%;"/>
 

In [36]:
search_emb(database(G, 276649))

  0.005140 seconds (5 allocations: 4.297 KiB)


<div><div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://tse1-2.mm.bing.net/th?id=OIP.c7Jo90ZDr3Xn1h_4CW15hQHaI4" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">1 - id: 276649 - w: -0.0</span>
        <span>picasso paintings of therese best 25 pablo picasso ideas on picasso</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://cdn.gallerix.asia/x/src/news/2018/Feb/annabelle.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">2 - id: 10106082 - w: 0.07</span>
        <span>&quot;&quot;&quot;&quot;&quot;&quot;&quot;Annabelle&quot;&quot;&quot;&quot; Picasso - renommer l&apos;image&quot;&quot;&quot;</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://arthive.net/res/media/img/orig/work/f82/190422.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">3 - id: 120995 - w: 0.09</span>
        <span>Pablo Picasso. Jacqueline with flowers</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://cdn.shopify.com/s/files/1/0030/0816/7981/products/C-0087_pablo_picasso_woman_hat_tube_XL_preview_2.jpg?v=1560457398" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">4 - id: 8241714 - w: 0.09</span>
        <span>Pablo Picasso - Woman in Hat and Fur Collar</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://www.galeriedada.com/img_big/Pic25.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">5 - id: 2912445 - w: 0.11</span>
        <span>Head of a Woman and Stars</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://images.squarespace-cdn.com/content/v1/53f353f4e4b0dbe1ff73ced2/1422512239182-DY3BI5Z51HWHD1JJ43SL/ke17ZwdGBToddI8pDm48kIkSVuU5FBEttbBmb1_WZWpZw-zPPgdn4jUwVcJE1ZvWEtT5uBSRWt4vQZAgTJucoTqqXjS3CfNDSuuf31e0tVF4v0VhzdbKbwhHmsdGUP7ETZpks4BHXsHNS63VxYA0K7vSAxncsgLUx_FOHuXVbfc/17+Picasso+Portrait+of+Dora+Maar+1937.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">6 - id: 7966224 - w: 0.11</span>
        <span>Portrait of Dora Maar, 1937</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://www.ownmodern.com/images/monet_rothko_kandinsky_more/51gPNqB2FDL._SY576_CR0,0,400,500_.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">7 - id: 5664127 - w: 0.11</span>
        <span>Pablo Picasso Portrait of Marie Therese Framed Oil Painting, 20 x 24</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://images.squarespace-cdn.com/content/57a4bc6303596ec41f3101a8/1503075806663-NLTF9EX1YPLZFUR9VOSE/Dora+Maar+by+Picasso%2C+1941?content-type=image%2Fjpeg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">8 - id: 4522674 - w: 0.11</span>
        <span>Dora Maar by Picasso, 1941</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://p1.liveauctioneers.com/546/23202/8178368_1_x.jpg?version=1289414275&amp;amp;width=170&amp;amp;format=pjpg&amp;amp;auto=webp&amp;amp;quality=50" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">9 - id: 5663288 - w: 0.11</span>
        <span>PABLO PICASSO The Red Armchair Print, Limited Edition</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://artlogic-res.cloudinary.com/w_1388,h_620,c_limit,f_auto,fl_lossy,q_auto/artlogicstorage/houkgallery/images/view/e74be8ca9d77c